## Video Referrence
- [Complete Tutorial on Vector Database - Learn ChromaDB, Pinecone & Weaviate | Generative AI](https://www.youtube.com/watch?v=8KrTO9bS91s)
- https://github.com/entbappy/Complete-Generative-AI-Course-on-YouTube/blob/main/Vector%20Database/2.Pinecone_demo.ipynb


## Import All the Required Libraries

In [6]:
from langchain.document_loaders import PyPDFDirectoryLoader, PyPDFLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os
import openai

In [2]:
from dotenv import load_dotenv

load_dotenv()

openAi_key = os.getenv('OPENAI_API_KEY')
pinecone_key = os.getenv('PINECONE_API_KEY')
openAi_base = os.getenv("OPENAI_API_BASE")

## Extract the Text from the PDF's

In [ ]:
def load_pdfs_from_folder(folder_path: str) -> list[Document]:
    all_docs: list[Document] = []
    for filename in os.listdir(folder_path):
        if not filename.lower().endswith(".pdf"):
            continue

        full_path = os.path.join(folder_path, filename)
        loader = PyMuPDFLoader(full_path)
        docs = loader.load()

        # Derive title from filename, e.g. "My Paper.pdf" -> "My Paper"
        title = os.path.splitext(filename)[0]

        # Add the title meta-field to each Document
        for doc in docs:
            # doc.metadata already contains things like 'source'; we just add 'Title'
            doc.metadata["Title"] = title

        all_docs.extend(docs)

    return all_docs


# ✅ Load all PDFs in the folder
data = load_pdfs_from_folder("website_content/")

In [ ]:
# loader = PyPDFDirectoryLoader("website_content")
# # loader = PyPDFLoader("Website_Report_V1.pdf")
# data = loader.load()

In [ ]:
data

## Clean Data

In [ ]:
import re

def clean_page(text: str) -> str:
    # Remove zero-width space characters
    text = text.replace('\u200b', '')
    
    # Remove artificial newlines that break up sentences or words
    text = re.sub(r'\n+', ' ', text)             # Merge multiple newlines into one space
    text = re.sub(r'(?<=\w)-\s+(?=\w)', '', text) # Fix hyphenated line breaks (e.g., "subsi-\ndy" → "subsidy")
    text = re.sub(r'\s+', ' ', text)             # Normalize extra spaces
    return text.strip()

def clean_documents(docs: list[Document]) -> list[Document]:
    cleaned_docs = []
    for doc in docs:
        cleaned_text = clean_page(doc.page_content)
        cleaned_docs.append(Document(page_content=cleaned_text, metadata=doc.metadata))
    return cleaned_docs


In [ ]:
clean_data = clean_documents(data)
print(clean_data)

In [ ]:
print(clean_data[0])

## Split the Extracted Data into Text Chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(clean_data)

In [ ]:
print(len(text_chunks))
text_chunks[0]

## Load OPENAI API

In [ ]:
# os.environ['OPENAI_API_KEY'] = 

## Download the Embeddings

In [3]:
# Use a specific embedding model
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# text = "LangChain is an AI framework for LLMs."
# vector = embeddings_model.embed_query(text)

# print(len(vector))
# print(vector[:5])  # Print first 5 values for readability


## Initializing the Pinecone

In [4]:


# # os.environ['PINECONE_API_KEY'] = "pcsk_7HmYTn_KS4n9fp4CzxTjTrKpYWaaBgHvP2JPRRx9fp5URALDkKuCC1yeZYhbZ557rEfjYT"
# # pc = pinecone.Pinecone(os.getenv('PINECONE_API_KEY'))
# pc = pinecone.Pinecone(pinecone_key)
index_name = "fit5120-tm01"
# index = pc.Index(index_name)

## Create Embeddings for each of the Text Chunk

In [ ]:
# # Pinecone.from_texts() vs Pinecone.from_documents()
# # .from_documents() stores meta data while .from_texts() does not

# docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings_model, index_name=index_name)

## If you already have an index, you can load it like this

In [5]:
docsearch = Pinecone.from_existing_index(index_name, embeddings_model)
docsearch

PineconeConfigurationError: You haven't specified an Api-Key.

## Similarity Search

In [ ]:
query = "How much is the diesel subisdy expenditure in 2024"

docs = docsearch.similarity_search(query, k=3)

In [ ]:
docs

## Creating a LLM Model Wrapper

In [ ]:
llm_gpt4 = ChatOpenAI(model="gpt-4.1-nano") #gpt-4.1-mini

qa = RetrievalQA.from_chain_type(llm=llm_gpt4, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={"k": 3}))

In [ ]:
qa

## Q/A

In [ ]:
query = "How much was the government expenditure?"
qa.invoke(query)

In [ ]:
query = "How much is the diesel subisdy expenditure in 2024"
qa.invoke(query)

In [ ]:
query = "How much is the diesel subsidy expenditure in 2024"
qa.invoke(query)

In [ ]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    break
  if user_input == '':
    continue
  result = qa.invoke({'query': user_input})
  print(f"Answer: {result['result']}")
     

### Test Proxy

In [8]:
import requests

# Replace this with your actual Cloudflare Worker URL
PROXY_BASE_URL = "https://policylensai.wanningc11.workers.dev"

In [9]:
embedding_url = f"{PROXY_BASE_URL}/v1/embeddings"

headers = {
    "Content-Type": "application/json"
}

payload = {
    "model": "text-embedding-3-small",  # match what your backend expects
    "input": "Test embedding input from Jupyter"
}

response = requests.post(embedding_url, headers=headers, json=payload)

print("Status Code:", response.status_code)
print("Response JSON:", response.json())

Status Code: 200
Response JSON: {'object': 'list', 'data': [{'object': 'embedding', 'index': 0, 'embedding': [-0.040278684, -0.03507916, 0.01236358, -0.028703269, -0.015998542, -0.037525993, 0.017386653, -0.003931995, 0.027926868, 0.018233633, 0.01748076, -0.008005153, 0.0032967585, -0.015363307, -0.03387927, -0.03272643, 0.016504379, 0.05401861, -0.037149556, 0.07058181, -0.0040025767, -0.006328835, 0.008469817, -0.0121283075, 0.059335772, -0.04618403, -0.0150456885, 0.01022848, 0.02463305, 0.023056723, -0.013104688, -0.026750505, -0.052089375, 0.03514974, -0.01679847, 0.0020689291, 0.016080888, 0.02145687, -0.026397595, 0.03507916, 0.0050583626, -0.014610434, 0.003834945, 0.063711844, 0.0015689747, 0.015822088, -0.036490794, -0.03496152, -0.019833488, 0.03910232, -0.012269471, -0.015527997, -0.0012866475, -0.018892396, -0.002829154, 0.045007665, -0.022068579, 0.042325556, -0.003023254, -0.044207737, -0.012610616, -0.023656668, -0.011351908, 0.0371025, 0.025621196, -0.03482036, -0.007

In [10]:
chat_url = f"{PROXY_BASE_URL}/v1/chat/completions"

chat_payload = {
    "model": "gpt-3.5-turbo",  # or gpt-4 if you're using that in backend
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    "temperature": 0.7
}

chat_response = requests.post(chat_url, headers=headers, json=chat_payload)

print("Status Code:", chat_response.status_code)
print("Response JSON:", chat_response.json())


Status Code: 200
Response JSON: {'id': 'chatcmpl-BZ9hWBYMjalG7vu5pQj1Ezm21Y7gl', 'object': 'chat.completion', 'created': 1747718470, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The capital of France is Paris.', 'refusal': None, 'annotations': []}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 24, 'completion_tokens': 7, 'total_tokens': 31, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'service_tier': 'default', 'system_fingerprint': None}
